In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import random
from PIL import Image
from datetime import datetime
import unicodedata
import nltk
from wordcloud import WordCloud

In [3]:
customer=pd.read_csv(r"C:\Users\AMI\Desktop\ineu\Ecom\olist_customers_dataset.csv")
loc=pd.read_csv(r"C:\Users\AMI\Desktop\ineu\Ecom\olist_geolocation_dataset.csv")
loc.rename(columns={"geolocation_zip_code_prefix":"zip_code_prefix"},inplace=True)
order_items=pd.read_csv(r"C:\Users\AMI\Desktop\ineu\Ecom\olist_order_items_dataset.csv")
order_pay=pd.read_csv(r"C:\Users\AMI\Desktop\ineu\Ecom\olist_order_payments_dataset.csv")
order_review=pd.read_csv(r"C:\Users\AMI\Desktop\ineu\Ecom\olist_order_reviews_dataset.csv")
orders=pd.read_csv(r"C:\Users\AMI\Desktop\ineu\Ecom\olist_orders_dataset.csv")
products=pd.read_csv(r"C:\Users\AMI\Desktop\ineu\Ecom\olist_products_dataset.csv")
sellers=pd.read_csv(r"C:\Users\AMI\Desktop\ineu\Ecom\olist_sellers_dataset.csv")
sellers.rename(columns={"seller_zip_code_prefix":"zip_code_prefix"},inplace=True)
product_name=pd.read_csv(r"C:\Users\AMI\Desktop\ineu\Ecom\product_category_name_translation.csv")

In [4]:
df = pd.merge(orders,order_pay, on="order_id")
df = df.merge(customer, on="customer_id")
df = df.merge(order_items, on="order_id")
df = df.merge(products, on="product_id")
df = df.merge(product_name, on="product_category_name")
df = df.merge(order_review, on="order_id")
df = df.merge(sellers,on="seller_id")
df= df.merge(loc,on="zip_code_prefix")
df.head()
df.shape


(16849468, 44)

In [5]:
df.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,...,review_comment_message,review_creation_date,review_answer_timestamp,zip_code_prefix,seller_city,seller_state,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,credit_card,...,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,9350,maua,SP,-23.680114,-46.452454,maua,SP
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,credit_card,...,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,9350,maua,SP,-23.675223,-46.441038,maua,SP
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,credit_card,...,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,9350,maua,SP,-23.685712,-46.444796,mauá,SP


In [5]:
df["order_approved_at"].fillna(df["order_purchase_timestamp"], inplace=True)
df["order_delivered_customer_date"].fillna(df["order_estimated_delivery_date"], inplace=True)

#dropping order delivery carrier date
df.drop(labels='order_delivered_carrier_date',axis=1,inplace=True)

In [6]:
df['review_creation_date'] = pd.to_datetime(df['review_creation_date'], format='%Y/%m/%d %H:%M:%S')
df['review_answer_timestamp'] = pd.to_datetime(df['review_answer_timestamp'], format='%Y/%m/%d %H:%M:%S')
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'], format='%Y/%m/%d %H:%M:%S')
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'], format='%Y/%m/%d %H:%M:%S')
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'], format='%Y/%m/%d %H:%M:%S')
df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'], format='%Y/%m/%d %H:%M:%S')



In [7]:
df['delivery_time'] = (df['order_delivered_customer_date']- df['order_purchase_timestamp'])

def hours(dt):
    hours=dt.total_seconds() / 3600
    return hours
df['delivery_time']=df['delivery_time'].apply(hours)


In [8]:
df.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,payment_installments,...,review_creation_date,review_answer_timestamp,zip_code_prefix,seller_city,seller_state,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,delivery_time
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-10 21:25:13,2017-10-18,1,credit_card,1,...,2017-10-11,2017-10-12 03:43:48,9350,maua,SP,-23.680114,-46.452454,maua,SP,202.477778
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-10 21:25:13,2017-10-18,1,credit_card,1,...,2017-10-11,2017-10-12 03:43:48,9350,maua,SP,-23.675223,-46.441038,maua,SP,202.477778
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-10 21:25:13,2017-10-18,1,credit_card,1,...,2017-10-11,2017-10-12 03:43:48,9350,maua,SP,-23.685712,-46.444796,mauá,SP,202.477778


In [12]:
columns=["order_id","customer_id","order_purchase_timestamp","order_approved_at","order_delivered_customer_date","payment_sequential","review_answer_timestamp"]

df.drop(columns,axis=1,inplace=True)


KeyError: "['order_id' 'customer_id' 'order_purchase_timestamp' 'order_approved_at'\n 'order_delivered_customer_date' 'payment_sequential'\n 'review_answer_timestamp'] not found in axis"

In [13]:
df.drop("customer_unique_id",axis=1,inplace=True)

In [14]:
columns=["order_item_id","review_comment_message","review_creation_date","seller_state"]
df.drop(columns,axis=1,inplace=True)

In [15]:
df.head(3)

,order_status,order_estimated_delivery_date,payment_type,payment_installments,payment_value,customer_zip_code_prefix,customer_city,customer_state,product_id,seller_id,...,review_id,review_score,review_comment_title,zip_code_prefix,seller_city,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,delivery_time
0,delivered,2017-10-18,credit_card,1,18.12,3149,sao paulo,SP,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,...,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,9350,maua,-23.680114,-46.452454,maua,SP,202.477778
1,delivered,2017-10-18,credit_card,1,18.12,3149,sao paulo,SP,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,...,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,9350,maua,-23.675223,-46.441038,maua,SP,202.477778
2,delivered,2017-10-18,credit_card,1,18.12,3149,sao paulo,SP,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,...,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,9350,maua,-23.685712,-46.444796,mauá,SP,202.477778
